In [ ]:
import os
import cv2
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import normalize
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense

# Set the directory containing the dataset
image_directory = 'C:/Users/jaafa/Downloads/Converted Dataset/Coronary artery disease Model'

# Lists the 'yes' and 'no' folders
NO_CAD = os.listdir(image_directory + 'no/')
YES_CAD = os.listdir(image_directory + 'yes/')

dataset = []
label = []
input_size = 64

# Common image extensions
valid_image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif']

# Debug: Print files in 'yes' and 'no' directories
print("Files in 'yes/' directory:", YES_CAD)
print("Files in 'no/' directory:", NO_CAD)

# Function to process images in directories
def process_images_from_directory(directory_path, label_value):
    global dataset, label
    for root, dirs, files in os.walk(directory_path):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            print(f"Found file: {file_name} in {root}")

            # Check if file has a valid image extension
            if not any(file_name.lower().endswith(ext) for ext in valid_image_extensions):
                print(f"File is not an image (invalid extension): {file_path}")
                continue  # Skip non-image files

            # Try reading the image with cv2 to ensure it's an image
            try:
                image = cv2.imread(file_path)
                if image is not None:  # Valid image
                    print(f"Processing image: {file_path}")
                    image = Image.fromarray(image, 'RGB')
                    image = image.resize((input_size, input_size))
                    dataset.append(np.array(image))
                    label.append(label_value)
                else:
                    print(f"File is not a valid image (cv2 read failed): {file_path}")
            except Exception as e:
                print(f"Error processing file {file_name}: {e}")

# Process 'yes' images (label = 1)
for subject_dir in YES_CAD:
    subject_path = os.path.join(image_directory + 'yes/', subject_dir)
    if os.path.isdir(subject_path):  # Ensure it's a directory
        print(f"Found directory in 'yes/' folder: {subject_dir}")
        process_images_from_directory(subject_path, label_value=1)

# Process 'no' images (label = 0)
for subject_dir in NO_CAD:
    subject_path = os.path.join(image_directory + 'no/', subject_dir)
    if os.path.isdir(subject_path):  # Ensure it's a directory
        print(f"Found directory in 'no/' folder: {subject_dir}")
        process_images_from_directory(subject_path, label_value=0)

# Print the number of images and labels processed
print(f"Processed images: {len(dataset)}")
print(f"Processed labels: {len(label)}")

# Check if dataset and labels are not empty
if len(dataset) > 0 and len(label) > 0:
    dataset = np.array(dataset)
    label = np.array(label)
    x_train, x_test, y_train, y_test = train_test_split(dataset, label, test_size=0.2)
    x_train = normalize(x_train, axis=1)
    x_test = normalize(x_test, axis=1)
else:
    print("Dataset or labels are empty. Check the image loading process.")

# Build the model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(input_size, input_size, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, batch_size=16, epochs=15, validation_data=(x_test, y_test), shuffle=False)

# Save the model
model.save('CAD_trains.h5')
